In [ ]:
from app.Labeler import Labeler
import pandas as pd
from app.device_namespace import SNESORS_CSV
from app.labels_namespace import ORIENTATION, OTHER
from app.utils import load_data_for_device
import copy

In [ ]:
current_date = "2024-02-25"
labeler = Labeler(date=current_date)

In [ ]:
df_dict = load_data_for_device(device_names=list(labeler.folders_dict.keys()),current_date=current_date)

In [ ]:
df_dict['HONOR_8X']

In [ ]:
honor_data = copy.deepcopy(df_dict['HONOR_8X'])

In [ ]:
import math

def scale_angle(angle):
    scaled_angle = (angle + 2 * math.pi) % (2 * math.pi)
    return scaled_angle

def data_transformation(dataframe : pd.DataFrame, smoothing = True, scaling = True):
    """applay transformations such as scaling angels and smoothing (for now only to the orientation data)"""
    large_dict_dict = {}
    for key in list(dataframe.keys()):
        if key == 'time':
            continue
        elif key == 'Orientation':
            temp = pd.DataFrame(dataframe['Orientation'].tolist(), columns=ORIENTATION)
            #scaling
            if scaling:
                temp = temp.applymap(scale_angle)

            #normalizacja
            #temp = temp.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
            if smoothing:
                temp = temp.rolling(window=3, min_periods=1).mean()

        else:
            temp = pd.DataFrame(dataframe[key].tolist(), columns=OTHER)


        temp['time'] = honor_data['time']

        large_dict_dict[key] = temp
    return large_dict_dict


In [ ]:
from app.DataTransformer import DataTransformer

#smoothed and scaled data
honor_smooth_scaled = data_transformation(honor_data)
final_smooth_scaled = DataTransformer.prepare_final_df(device_dataframes = honor_smooth_scaled)

#smoothed and regular data
honor_scaled = data_transformation(honor_data, smoothing=False)
final_scaled = DataTransformer.prepare_final_df(device_dataframes = honor_scaled)

#regular data
honor_regular = data_transformation(honor_data, smoothing=False, scaling=False)
final_regular = DataTransformer.prepare_final_df(device_dataframes = honor_regular)


In [ ]:
from app.visualization_utils import draw_plot


In [ ]:
final_smooth_scaled

In [ ]:
draw_plot(final_smooth_scaled, 'Orientation')

In [ ]:
draw_plot(final_scaled, 'Orientation')

In [ ]:
draw_plot(final_regular, 'Orientation')

In [ ]:
#visualization next part -
----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd

def select_data_range(start: str, end: str, df: pd.DataFrame):
    """Select a range of data from a DataFrame based on the start and end times."""
    start_time = pd.to_datetime(start).time()
    end_time = pd.to_datetime(end).time()

    #throw an exception when the start time is greater than the end time
    if start_time > end_time:
        raise ValueError("The start time cannot be greater than the end time.")


    # Select data between start_time and end_time
    return df.between_time(start_time, end_time)


def draw_plot_plt(df: pd.DataFrame, start: str, end: str,  col: str, output:str, window_size: int = 100):
    """Function to create a plot for each value in a given column over time."""
    df_copy = df.copy()
    df_copy = select_data_range(df=df_copy, start=start, end=end)
    if "time" not in df_copy.columns:
        df_copy.reset_index(inplace=True)
    df_split = df_copy[col].apply(pd.Series)

    labels = ORIENTATION if col == "Orientation" else OTHER

    fig, ax = plt.subplots(figsize=(10, 6))

    # Set the y-axis limits
    ax.set_ylim(df_split.min().min(), df_split.max().max())

    def update(frame):
        ax.clear()
        for i, (cord, label) in enumerate(zip(df_split.columns, labels)):
            if col == "Orientation" and i < 4:
                continue
            ax.plot(
                df_copy["time"].iloc[: frame + 1],
                df_split[cord].iloc[: frame + 1],
                label=f"{label}",
            )
        ax.set_title(f"{col} over time")
        ax.set_xlabel("time")
        ax.set_ylabel(col)
        ax.legend()

        # Keep the y-axis limits fixed
        ax.set_ylim(df_split.min().min(), df_split.max().max())

        # Set the x-axis limits to create a moving window of window_size seconds
        if frame > window_size:
            ax.set_xlim(
                df_copy["time"].iloc[frame - window_size], df_copy["time"].iloc[frame]
            )

    duration = (df_copy["time"].iloc[-1] - df_copy["time"].iloc[0]).total_seconds()

    frames = len(df_copy)

    # Calculate the duration of one frame of the animation
    if frames > 1:
        frame_duration = (
            df_copy["time"].iloc[1] - df_copy["time"].iloc[0]
        ).total_seconds()
    else:
        frame_duration = duration

    # Prepare the animation
    ani = FuncAnimation(
        fig, update, frames=frames, repeat=False, interval=frame_duration * 1000
    )

    # Save the animation to a GIF file
    file_name = f"{col}_animation.gif"
    if os.path.exists(file_name):
        index = 1
        while os.path.exists(f"{col}_animation_{index}.gif"):
            index += 1
        file_name = f"{col}_animation_{index}.gif"

    ani.save(file_name, writer="pillow", fps=1 / frame_duration)

    plt.close()
    gif_to_mp4(input=output_path, output=output_path.replace('.gif', '.mp4'))
    print(
        f"Animation saved as {file_name}. Please open the file to view the animation."
    )


In [ ]:
draw_plot_plt(df = final_smooth_scaled, start= "12:44:56", end="12:45:20", col = 'Orientation')
#draw_plot_plt(df = final_smooth_scaled[65000:66000], col = 'Orientation')

In [ ]:
import imageio
import numpy as np

#Create reader object for the gif

def merge_gifs(gif1_path, gif2_path, output_path):
    # Create reader objects for the gifs
    gif1 = imageio.get_reader(gif1_path)
    gif2 = imageio.get_reader(gif2_path)

    # If they don't have the same number of frames take the shorter
    number_of_frames = min(gif1.get_length(), gif2.get_length())
    print(f'number_of_frames: {number_of_frames}, gif1.get_length(): {gif1.get_length()}, gif2.get_length(): {gif2.get_length()}')
    # Create writer object
    new_gif = imageio.get_writer(output_path)

    for frame_number in range(number_of_frames-1):
        img1 = gif1.get_next_data()
        img2 = gif2.get_next_data()
        # Merge images horizontally
        new_image = np.hstack((img1, img2))
        new_gif.append_data(new_image)

    gif1.close()
    gif2.close()
    new_gif.close()
    gif_to_mp4(input=output_path, output=output_path.replace('.gif', '.mp4'))

# Example usage:
merge_gifs('Orientation_animation_20.gif', 'Orientation_animation_21.gif', 'outputt.gif')

In [ ]:
def gif_to_mp4(input: str, output: str) -> None:
    """Convert a GIF file to an MP4 file."""
    ff = ffmpy.FFmpeg(
    inputs={input: None},
    outputs={output: None}
)
    ff.run()

In [ ]:
import ffmpy
ff = ffmpy.FFmpeg(
    inputs={'output.gif': None},
    outputs={'output.mp4': None}
)
ff.run()

In [ ]:
ff = ffmpy.FFmpeg(
    inputs={'output.mp4': None},
    outputs={'outputtemp.gif': None}
)
ff.run()

In [ ]:
def cut_video(start:str, end:str, input_path:str, output_path:str):
    ff = ffmpy.FFmpeg(
        inputs={input_path: None},
        outputs={output_path: f'-ss {start} -to {end}'}
    )
    ff.run()

In [ ]:
cut_video('00:00:00', '00:00:10', 'output.mp4', 'output_cut.mp4')

In [ ]:
ff = ffmpy.FFmpeg(
    inputs={'output_cut.mp4': None},
    outputs={'output_cut.gif': None}
)
ff.run()

In [ ]:
from PIL import Image

def merge_gifs(gif1_path, gif2_path, output_path):
    """Combine two GIFs side by side."""
    # Create reader objects for the gifs
    gif1 = imageio.get_reader(gif1_path)
    gif2 = imageio.get_reader(gif2_path)

    # If they don't have the same number of frames take the shorter
    number_of_frames = min(gif1.get_length(), gif2.get_length())
    print(f'number_of_frames: {number_of_frames}, gif1.get_length(): {gif1.get_length()}, gif2.get_length(): {gif2.get_length()}')

    # Create writer object
    new_gif = imageio.get_writer(output_path)

    for frame_number in range(number_of_frames-1):
        img1 = gif1.get_next_data()
        img2 = gif2.get_next_data()

        # Resize the images
        img1 = Image.fromarray(img1).resize((200, 200))
        img2 = Image.fromarray(img2).resize((200, 200))

        # Ensure both images have the same number of channels
        if img1.mode != img2.mode:
            if img1.mode == 'RGBA':
                img1 = img1.convert('RGB')
            else:
                img2 = img2.convert('RGB')

        # Merge images horizontally
        new_image = np.hstack((np.array(img1), np.array(img2)))
        new_gif.append_data(new_image)

    gif1.close()
    gif2.close()
    new_gif.close()

# Example usage:
#merge_gifs('Orientation_animation_20.gif', 'output_cut.gif', 'output_merged.gif')

In [ ]:
merge_gifs('Orientation_animation_20.gif', 'output_cut.gif', 'output_merged.gif')

In [ ]:
ff = ffmpy.FFmpeg(
    inputs={'output_merged.gif': None},
    outputs={'output_merged.mp4': '-r 24 -c:v libx264'}
)
ff.run()

In [ ]:
# Define the input files
input_file1 = 'output.mp4'
input_file2 = 'output_cut.mp4'

# Define the output file
output_file = 'merged_videos.mp4'

# Construct the FFmpeg command
ff = ffmpy.FFmpeg(
    inputs={input_file1: None, input_file2: None},
    outputs={output_file: '-filter_complex "[0:v][1:v]vstack=inputs=2[v]" -map "[v]"'}
)

# Run FFmpeg command
ff.run()


In [ ]:

splitLength = 5
for i in range(int(videoLength/splitLength)):
    start =i*60
    length=splitLength*60
    os.system("ffmpeg -i source-file.foo -ss " + str(start) + " -t " + str(length) + " clip"+str(i)+".m4v")

In [ ]:
from moviepy.editor import VideoFileClip

def gif_to_mp4(gif_path, mp4_path, fps=10):
    # Wczytaj plik GIF
    clip = VideoFileClip(gif_path)

    # Ustaw fps, jeśli nie został wcześniej ustawiony
    if clip.fps is None:
        clip = clip.set_fps(fps)

    # Zapisz jako plik MP4
    clip.write_videofile(mp4_path, codec='libx264')

    # Zwolnij zasoby
    clip.close()

# Przykładowe użycie funkcji
gif_to_mp4('output.gif', 'output.mp4')

In [ ]:
out = "combined.gif"

In [ ]:
out[:-3] + 'mp4'

In [ ]:
copy___ = final_smooth_scaled.copy()

In [ ]:
type(final_smooth_scaled.index[0])

In [ ]:
def select_data_range(start: str, end: str, df: pd.DataFrame):
    """Select a range of data from a DataFrame based on the start and end timestamps."""
    start_time = pd.Timestamp(start)
    end_time = pd.Timestamp(end)

    return df[(df["time"] >= start_time) & (df["time"] <= end_time)]

In [ ]:
def select_data_range(start: str, end: str, df: pd.DataFrame):
    """Select a range of data from a DataFrame based on the start and end times."""
    start_time = pd.to_datetime(start).time()
    end_time = pd.to_datetime(end).time()

    # Select data between start_time and end_time
    return df.between_time(start_time, end_time)


In [ ]:
final_smooth_scaled

In [ ]:
select_data_range("12:00:00", "12:00:01", final_smooth_scaled)

In [ ]:
final_smooth_scaled.loc['2024-02-25 10:57:19.100000' : '2024-02-25 10:57:19.700000']

In [ ]:
final_smooth_scaled[65000:65200]

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def draw_plot_with_marker(df: pd.DataFrame, col: str, video_time: float):
    df_copy = df.copy()
    if "time" not in df_copy.columns:
        df_copy.reset_index(inplace=True)
    df_split = df_copy[col].apply(pd.Series)

    fig = go.Figure()

    labels = ORIENTATION if col == "Orientation" else OTHER

    for i, (cord, label) in enumerate(zip(df_split.columns, labels)):
        if col == "Orientation" and i < 4:
            continue
        fig.add_trace(
            go.Scatter(
                x=df_copy["time"], y=df_split[cord], mode="lines", name=f"{label}"
            )
        )

    # Dodanie kreski wskazującej na wybrany czas w filmie
    fig.add_trace(
        go.Scatter(
            x=[video_time, video_time],
            y=[df[col].min(), df[col].max()],
            mode="lines",
            line=dict(color="red", width=2, dash="dash"),
            name="Video Time",
        )
    )

    fig.update_layout(
        title=f"{col} over time",
        xaxis_title="Time",
        yaxis_title=col,
        showlegend=True,
    )

    fig.show()


In [ ]:
large_dict_dict = {}
for key in list(honor_data.keys()):
    if key == 'time':
        continue
    elif key == 'Orientation':
        temp = pd.DataFrame(honor_data['Orientation'].tolist(), columns=ORIENTATION)
        #temp = temp.applymap(scale_angle)

        #normalizacja
        #temp = temp.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

    else:
        temp = pd.DataFrame(honor_data[key].tolist(), columns=OTHER)
    temp = temp.rolling(window=3, min_periods=1).mean()

    temp['time'] = honor_data['time']

    large_dict_dict[key] = temp


In [ ]:
from app.DataTransformer import DataTransformer
final_regular = DataTransformer.prepare_final_df(device_dataframes = large_dict_dict)

In [ ]:
from app.visualization_utils import draw_plot

In [ ]:
import plotly.graph_objects as go

def draw_plot(df: pd.DataFrame, col: str):
    """function that make a plot for each value in given column over time column
    example: TotalAcceleration col has values given as a list of 3 like [9.5183687210083, -1.0018105506896973, 0.62888
    it should generate a 3 plots with value over time"""
    df_copy = df.copy()
    #df_copy["time"] = pd.to_datetime(df_copy["time"])
    if 'time' not in df_copy.columns:
        df_copy.reset_index(inplace=True)
    # df_copy[col] = df_copy[col].str.replace("nan", "0").apply(ast.literal_eval)
    df_split = df_copy[col].apply(pd.Series)
    # df_split = df_split.rename(columns={0: 'X', 1: 'Y', 2: 'Z'})

    fig = go.Figure()

    labels = ORIENTATION if col == "Orientation" else OTHER

    for i, (cord, label) in enumerate(zip(df_split.columns, labels)):
        if col == "Orientation" and i < 4:
            continue
        fig.add_trace(
            go.Scatter(
                x=df_copy["time"], y=df_split[cord], mode="lines", name=f"{label}"
            )
        )

    fig.update_layout(title=f"{col} over time", xaxis_title="time", yaxis_title=col)

    fig.show()


In [ ]:
#scaled angels
draw_plot(final, "Orientation")

In [ ]:
#unscaled angels
draw_plot(final2, "Orientation")

In [ ]:
#normalized
draw_plot(final, "Orientation")

In [ ]:
draw_plot(honor_data, 'Orientation')

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import cv2

def create_animation(df: pd.DataFrame, col: str, video_path: str, start_time: float, end_time: float):
    df_copy = df.copy()
    if "time" not in df_copy.columns:
        df_copy.reset_index(inplace=True)
    df_split = df_copy[col].apply(pd.Series)

    # Otwórz film za pomocą OpenCV
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Utwórz obiekt VideoWriter do zapisywania animacji
    out = cv2.VideoWriter('animation.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (2 * frame_width, frame_height))

    fig = go.Figure()

    labels = ["X", "Y", "Z"]

    for cord, label in enumerate(labels):
        fig.add_trace(
            go.Scatter(
                x=df_copy["time"], y=df_split[cord], mode="lines", name=f"{label}"
            )
        )

    fig.update_layout(
        title=f"{col} over time",
        xaxis_title="Time",
        yaxis_title=col,
        showlegend=True,
        width=frame_width,
        height=frame_height
    )

    frame_number = 0
    start_frame = start_time * 30  # Przelicz czas na liczbę klatek (przy 30 FPS)
    end_frame = end_time * 30

    while True:
        ret, frame = cap.read()
        if not ret or frame_number >= end_frame:
            break

        if frame_number >= start_frame:
            # Ustaw bieżący czas w filmie
            video_time = frame_number * (1 / 30)  # Czas klatki w sekundach (przy 30 FPS)

            # Przygotuj wykres z aktualnym czasem w filmie
            fig.update_traces(
                go.Scatter(x=[video_time, video_time], y=[df[col].min(), df[col].max()], mode="lines", line=dict(color="red", width=2, dash="dash"), name="Video Time")
            )

            # Zapisz wykres jako obraz
            fig.write_image(f"frame_{frame_number}.png")

            # Połącz obraz wykresu i klatkę filmu
            combined_frame = cv2.imread(f"frame_{frame_number}.png")
            combined_frame[:, frame_width:] = frame

            # Zapisz łączną klatkę do pliku wideo
            out.write(combined_frame)

        frame_number += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Przykładowe wywołanie funkcji
create_animation(df, "Orientation", "movie.mp4", 30, 60)

In [ ]:
pitch - ? ostre zakrety na krawędzi
yaw -

In [ ]:
#normalization
final["Orientation"]

In [ ]:
------------------------------